In [53]:
import random

# StringIO behaves like a file object 
from io import StringIO 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

import copy

%matplotlib inline 

In [54]:
class CqkProblem:
    def __init__(self, r, n, d, a, b, up):
        self.n = n
        self.r = r
        self.d = list(d)
        self.a = list(a)
        self.b = list(b)
        self.up = list(up)

In [55]:
def generate_cqk_problem(n):
    d = []
    low = []
    up = []
    b = []
    a = []
    temp = 0
    lb = 0.0
    ub = 0.0
    lower = 10
    upper = 25
    r = 0

    for i in range(n):
        
        b.append(10 + 14*random.random())
        up.append(1 + 14*random.random())
        
        ub = ub + b[i]*up[i];
        
        #Uncorrelated
        d.append(random.randint(10,25))
        a.append(random.randint(10,25))
        
    r = ub*0.7;
    
    return CqkProblem( r, n, d, a, b, up)

In [56]:

def initial_lambda(p, lamb):
    s0=0.0
    q0=0.0
    slopes = []
    for i in range(p.n):
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]
    lamb = (p.r-s0)/q0
    return lamb, slopes

In [57]:
def phi_lambda(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [58]:
def phi_lambda_two(p,lamb1,r):
    
    slopes = []
    
    deriv1 = 0
    phi1 = r * -1

    x2 = []
    x1 = []
    
    for i in range(p.n):
        
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        
        x1.append( (p.b[i] * lamb1 + p.a[i])/p.d[i])

        if x1[i] > p.up[i]:
            x1[i] = p.up[i]
        else:
            deriv1 = deriv1 + slopes[i];
        phi1 = phi1 + p.b[i] * x1[i];
        

            
    return deriv1,phi1

In [59]:
MAX_IT = 20
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton(p):
    lambs = [] 
    phis = []
    derivs = []
    phi = 0
    lamb = 0
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
#     lamb1 = 5.52
#     deriv1,phi1 = phi_lambda_two(p,lamb1,r)
    
    lamb, slopes = initial_lambda(p,lamb)
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    derivs.append(deriv)
    phis.append(phi)
    it = 1
#     print(it, deriv, phi,lamb)
    negativo = False
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 1e-10:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            if it == 1:
                negativo = True
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 1e-10:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        
    if phi == 0.0:
        return it,lambs, derivs, phis,slopes,phi1
    elif alfa == beta:
        return -1,lambs, derivs, phis,slopes,phi1
    else:
        return -2,lambs, derivs, phis,slopes,phi1

In [163]:
MAX_IT = 20
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton_new_lamb(p,lamb,phi,deriv,slopes):
    lambs = [] 
    phis = []
    derivs = []
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    derivs.append(deriv)
    phis.append(phi)
    it = 1
#     print(it, deriv, phi,lamb)
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        
    if phi == 0.0:
        return it,lambs, derivs, phis,slopes
    elif alfa == beta:
        return -1,lambs, derivs, phis,slopes
    else:
        return -2,lambs, derivs, phis,slopes

In [60]:
def func_ty(p):
    t = np.arange(0, 15, 0.5).tolist()
    y = copy.deepcopy(t)
    for j in range(len(t)):
        lamb = t[j]

        soma = 0

        for i in range(n):
            soma = soma + p.b[i] * median(p.low[i], (p.b[i] *lamb + p.a[i]) / p.d[i], p.up[i])

        y[j] = soma
    return y

In [174]:
lista = []
for i in range(1000):
    n = 1000
    p = generate_cqk_problem(n)
    it,lambs, derivs, phis,slopes,phi1 = newton(p)
    soma_a = 0
    soma_b = 0
    soma_d = 0
    soma_up = 0
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_d += p.d[i]
        soma_up += p.up[i]
    
    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_d = soma_d/n
    soma_up = soma_up/n
    r = p.r/n
    sum_slopes = sum(slopes)
    if it > 2:
        l_rs = [phi1,lambs[-1]]
#         l_rs = [soma_a, soma_b, soma_d, r,lambs[0],lambs[1],lambs[2],lambs[3],lambs[-1]]


        lista.append(l_rs)

In [175]:
np.savetxt('instance_test.txt', lista, delimiter = ' ',newline='\n', fmt="%f")

In [176]:
c = ''
with open("instance_test.txt", "r") as fd:
    c = StringIO(fd.read())

In [177]:
d = c.read()
c = StringIO(d) 
d = np.loadtxt(c) 
feature_names = ['phi1','final_lamb']
# feature_names = ['a','b','d','r','initial_lamb','second_lamb','third_lamb','fourth_lamb','final_lamb']



In [178]:
knapsack = {"data":d, "feature_names": feature_names}
dataset = pd.DataFrame(knapsack['data'], columns = knapsack['feature_names'])

In [179]:
# Coletando x e y

X = dataset.iloc[:,:-1]
y = dataset['final_lamb'].values

In [180]:
dataset

,phi1,final_lamb
0,-625.987924,5.587579
1,-1888.403768,5.722995
2,-1042.519754,5.632295
3,1366.631950,5.373075
4,792.268623,5.437751
...,...,...
995,2319.734392,5.279063
996,674.451252,5.449832
997,-1155.202591,5.642949
998,-788.897743,5.601763


In [181]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [182]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [183]:
# Padronização
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [184]:
# Aplicando a padronização aos dados
X_train_p = scaler.transform(X_train)
X_test_p = scaler.transform(X_test)

In [185]:

# from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

In [186]:
# mlp = MLPClassifier(hidden_layer_sizes = (30,30,30))
regr = MLPRegressor(activation='identity',random_state=1, max_iter=5000).fit(X_train_p, y_train)

In [187]:
regr.fit(X_train_p, y_train)

MLPRegressor(activation='identity', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=5000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [188]:
regr.predict(X_test_p[:10])

array([5.61142963, 5.44637305, 5.56040144, 5.62900336, 5.23210031,
       5.48372454, 5.63426193, 5.62571078, 5.45043625, 5.58547191])

In [189]:
regr.score(X_test_p, y_test)

0.9990401494704526

In [190]:
y_test[:10]

array([5.607716, 5.444965, 5.558342, 5.628674, 5.237346, 5.484706,
       5.635623, 5.627639, 5.45087 , 5.58466 ])

In [221]:
%%time
n = 1000
pp = generate_cqk_problem(n)
itt,lambss, derivss, phiss,slopess,phi11 = newton(pp)

CPU times: user 56.3 ms, sys: 16.9 ms, total: 73.2 ms
Wall time: 77.9 ms


In [222]:
itt,lambss, phiss

(4,
 [4.093463052413391, 5.248027096449932, 5.376832157643845, 5.378840707264723],
 [-14163.45150754815,
  -1300.147029210405,
  -19.657047513155504,
  -2.076916416626773e-11])

In [225]:
%%time
deriv1,phi1 = phi_lambda_two(pp,5.52,pp.r)



result = regr.predict(scaler.transform(np.asarray([[phi1]])))[0]

itt,lambss, derivss, phiss,slopess = newton_new_lamb(pp,result,deriv1, phi1,slopess)


CPU times: user 8.92 ms, sys: 158 µs, total: 9.07 ms
Wall time: 9 ms


In [226]:
result, itt, lambss

(5.371928258362982,
 3,
 [5.371928258362982, 5.37881622402116, 5.378840707264727])